In [1]:
import json
import os
import re
import logging
from functools import reduce
from collections import OrderedDict
import pprint

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.model_selection import ParameterGrid

from nnattack.variables import auto_var
from params import (
    compare_nns,
    nn_k1_robustness,
    nn_k3_robustness,
    
    optimality,
    
    dt_robustness,
    rf_robustness,
)
from utils import set_plot, params_to_dataframe, get_result, write_to_tex, union_param_key

auto_var.set_variable_value('random_seed', 0)
auto_var.set_variable_value('ord', 'inf')
logging.basicConfig(level=0)

Using TensorFlow backend.


In [2]:
def result_latex_figs(exp_name, control_var, caption):
    control = ParameterGrid(control_var)
    ret = """
\\begin{figure}[ht!]
\\centering"""
    img_paths = []
    for i, g in enumerate(control):
        dataset, ord = g['dataset'], g['ord']
        img_path = f'./figs/{exp_name}_{dataset}_{ord}.eps'
        dataset = dataset.replace("_", " ")
        ret += """
\\subfloat[%s]{
    \\includegraphics[width=.45\\textwidth]{%s}}""" % (dataset, img_path)
        if i % 2 == 1:
            ret += "\n"
    ret += """
\\caption{%s}
\\label{fig:%s}
\\end{figure} 
""" % (caption, exp_name)
    return ret
                      
def plot_result(df, exp_nme, control_var, variables, show_plot=True):
    control = ParameterGrid(control_var)
    for g in control:
        title = exp_name
        temp_df = df
                      
        for k, v in g.items():
            if v in variable_name[k]:
                title = title + f"_{variable_name[k][v]}"
            else:
                title = title + f"_{v}"
            temp_df = temp_df.loc[df[k] == v]
                      
        fig, ax = plt.subplots()
        ax.set_title(title)
        for name, group in temp_df.groupby(variables):
            #print(name, len(group))
            eps_list = [re.findall(r'[-+]?\d*\.\d+|\d+', t)[0] for t in group.mean().index.tolist()[:-1]]
            s = [r for r in group.mean().tolist()[:-1] if not np.isnan(r)]
            x = [float(eps_list[i]) for i, r in enumerate(group.mean().tolist()[:-1]) if not np.isnan(r)]
                      
            if isinstance(name, str):
                if variables[0] not in variable_name:
                    label = name
                elif name in variable_name[variables[0]]:
                    label = variable_name[variables[0]][name]
                else:
                    label = name
            else:
                mod_names = []
                for i, n in enumerate(name):
                    if n in variable_name[variables[i]]:
                        mod_names.append(variable_name[variables[i]][n])
                    else:
                        mod_names.append(n)
                label = mod_name.join("_")
            ax.plot(x, s, label=label)

        dataset = g['dataset']
        ord = g['ord']
        set_plot(fig, ax)
        plt.savefig(f'./figs/{exp_name}_{dataset}_{ord}.eps', format='eps')
        plt.savefig(f'./figs/{exp_name}_{dataset}_{ord}.png', format='png')
        if show_plot:
            plt.show()
        else:
            plt.close()
                      
def get_avg(df, exp_nme, control_var, variables):
    control = ParameterGrid(control_var)
    for g in control:
        title = exp_name
        temp_df = df
                      
        for k, v in g.items():
            if v in variable_name[k]:
                title = title + f"_{variable_name[k][v]}"
            else:
                title = title + f"_{v}"
            temp_df = temp_df.loc[df[k] == v]
                      
        for name, group in temp_df.groupby(variables):
            print(name, len(group))
            eps_list = [re.findall(r'[-+]?\d*\.\d+|\d+', t)[0] for t in group.mean().index.tolist()[:-1]]
            s = [r for r in group.mean().tolist()[:-1] if not np.isnan(r)]
            x = [float(eps_list[i]) for i, r in enumerate(group.mean().tolist()[:-1]) if not np.isnan(r)]
                      
variable_name = {
    #'model': {
    #    ''
    #},
    'dataset': {
        'fashion_mnist35_2200_pca5': 'f-mnist35-pca5',
        'mnist35_2200_pca5': 'mnist35-pca5',
        'fashion_mnist06_2200_pca5': 'f-mnist06-pca5',
        'fashion_mnist35_2200_pca10': 'f-mnist35-pca10',
        'mnist35_2200_pca10': 'mnist35-pca10',
        'fashion_mnist06_2200_pca10': 'f-mnist06-pca10',
        'fashion_mnist35_2200_pca15': 'f-mnist35-pca15',
        'mnist35_2200_pca15': 'mnist35-pca15',
        'fashion_mnist06_2200_pca15': 'f-mnist06-pca15',
        
        'mnist17_2200_pca25': 'mnist17',
        'fashion_mnist35_2200_pca25': 'f-mnist35',
        'mnist35_2200_pca25': 'mnist35',
        'fashion_mnist06_2200_pca25': 'f-mnist06',
        'digits_pca5': 'digits-pca5',
        'digits_pca25': 'digits',
        'halfmoon_2200': 'halfmoon',
        
        'abalone': 'abalone',
        'iris': 'iris',
        'digits_pca5': 'digits-pca5',
        'wine': 'wine',
        
        'halfmoon_300': 'halfmoon',
        'fashion_mnist35_300_pca25': 'f-mnist35',
        'mnist35_300_pca25': 'mnist35',
        'fashion_mnist06_300_pca25': 'f-mnist06',
        
        'covtype_3200': 'covtype',
    },
    'attack': {
        'blackbox': 'Cheng\'s',
        'kernelsub_c10000_pgd': 'kernelsub',
        'kernelsub_c1000_pgd': 'kernelsub',
        'rev_nnopt_k1_20': 'nnopt-20-ori',
        'rev_nnopt_k1_50': 'nnopt-50-ori',
        'rev_nnopt_k1_20_region': 'nnopt-20',
        'rev_nnopt_k1_50_region': 'nnopt-50',
        'rev_nnopt_k3_20': 'nnopt-20-ori',
        'rev_nnopt_k3_50': 'nnopt-50-ori',
        'rev_nnopt_k3_20_region': 'nnopt-20',
        'rev_nnopt_k3_50_region': 'nnopt-50',
        'rev_nnopt_k5_20': 'nnopt-20-ori',
        'rev_nnopt_k5_50': 'nnopt-50-ori',
        'rev_nnopt_k5_20_region': 'nnopt-20',
        'rev_nnopt_k5_50_region': 'nnopt-50',
        'rev_nnopt_k7_20': 'nnopt-20-ori',
        'rev_nnopt_k7_50': 'nnopt-50-ori',
        'rev_nnopt_k7_20_region': 'nnopt-20',
        'rev_nnopt_k7_50_region': 'nnopt-50',
        
        'nnopt_k1_all': 'nnopt-all',
        'nnopt_k3_all': 'nnopt-all',
        
        'direct_k1': 'direct attack',
        'direct_k3': 'direct attack',
        'direct_k5': 'direct attack',
        'direct_k7': 'direct attack',
        
        'rf_attack_all': 'RF-all',
        'rf_attack_rev': 'RF-rev',
        'rf_attack_rev_20': 'RF-rev-20',
        'rf_attack_rev_50': 'RF-rev-50',
        'rf_attack_rev_100': 'RF-rev-100',
    },
    'ord': {},
}


In [14]:
def knn_attack_plots(exp_name, grid_param, caption='', show_plot=True):
    df = params_to_dataframe(grid_param)
    datasets = set.union(*[set(g['dataset']) for g in grid_param]) if isinstance(grid_param, list) else grid_param['dataset']

    control = {
        'dataset': datasets,
        'ord': grid_param[0]['ord'],
    }
    variables = ['attack']
    plot_result(df, exp_name, control, variables, show_plot)
    return result_latex_figs(exp_name, control, caption)
    
def get_var_name(var, arg):
    if var == 'model':
        if 'adv' in arg or 'robust' in arg:
            if 'd' in arg.split("_")[-1]:
                arg = "%s_%02d" % ("_".join(arg.split("_")[:-1]), int(arg.split("_")[-1][1:]))
            else:
                arg = "%s_%02d" % ("_".join(arg.split("_")[:-1]), int(arg.split("_")[-1]))
        else:
            arg = arg
    else:
        arg = variable_name[var].get(arg, arg)
    return arg.replace('_', '-')

def avg_pert_table(exp_name, grid_param, columns, rows, obj='avg_pert'):
    columns = list(filter(lambda a: a not in ['n_features', 'n_samples', 'n_classes'], columns))
    if len(columns) == 0 or len(rows) == 0:
        return pd.DataFrame({})
    df = params_to_dataframe(grid_param, 'avg_pert')
    
    temp_df = df.groupby(columns + rows)[obj].mean()
    temp_df_sem = df.groupby(columns + rows)[obj].sem()
    col_grid = OrderedDict({c: union_param_key(grid_param, c) for c in columns})
    row_grid = OrderedDict({r: union_param_key(grid_param, r) for r in rows})
    d = {}
                              
    for col in ParameterGrid(col_grid):
        col_k = tuple(col[c] for c in columns)
        col_name = tuple(get_var_name(c, col[c]) for c in columns)
        d[col_name] = {}
        for row in ParameterGrid(row_grid):
            row_k = tuple(row[r] for r in rows)
            row_name = tuple(get_var_name(r, row[r]) for r in rows)
            if (col_k + row_k) in temp_df:
                #d[col_name][row_name] = "$%.3f \pm %.3f$" % (temp_df[col_k + row_k], temp_df_sem[col_k + row_k])
                d[col_name][row_name] = "$%.3f$" % (temp_df[col_k + row_k])
                d[col_name][row_name] = d[col_name][row_name].replace("0.", ".")
            else:
                d[col_name][row_name] = -1

    d = {k: d[k] for k in sorted(d.keys())}
    return pd.DataFrame(d)

def dataset_stat_column(df, grid_param, columns, rows):
    if ("n_features" not in columns) and ("n_samples" not in columns) and ("n_classes" not in columns):
        return df
    
    column_names = {
        'n_features': '\# features',
        'n_samples': '\# examples',
        'n_classes': '\# classes',
    }
    
    d = df.to_dict(into=OrderedDict)
    datasets = union_param_key(grid_param, "dataset")
    if len(d.keys()) > 0:
        first_key = list(d.keys())[0]
        row_len = 1 if isinstance(d[first_key], str) else len(first_key)
        col_len = 1 if isinstance(first_key, str) else len(first_key)
        ori_cols = list(d.keys())
    else:
        row_len = 1
        col_len = 1
        ori_cols = []
    
    for dataset in datasets:
        X, y, _ = auto_var.get_var_with_argument("dataset", dataset)
        row_name = (get_var_name("dataset", dataset), )
        for col in columns:
            if col not in column_names:
                continue
            column_name = tuple(['-' for _ in range(col_len-1)] + [column_names[col]])
            if col == "n_features":
                d.setdefault(column_name, {})[row_name] = X.shape[1]
            elif col == "n_samples":
                d.setdefault(column_name, {})[row_name] = X.shape[0]
            elif col == "n_classes":
                d.setdefault(column_name, {})[row_name] = len(np.unique(y))
                
    for col in ori_cols:
        d.move_to_end(col)
        
    return pd.DataFrame(d)
    
    
def cmp_ratio(df):
    ret = OrderedDict()
    d = df.to_dict(into=OrderedDict)
    cmp_base = []
    
    for i, (col, col_dict) in enumerate(d.items()):
        ret[col] = col_dict
        if i == 0 or i == 1:
            cmp_base.append(col_dict)
            continue
        temp = {}
        for k, v in col_dict.items():
            if v == -1 or cmp_base[i % 2][k] == -1:
                temp[k] = int(-1)
            else:
                v = v.replace("$", "")
                t = cmp_base[i % 2][k].replace("$", "")
                temp[k] = "$%.2f$" % (float(v) / float(t))
        
        ret[tuple([c for c in col[:-1]] + ["%s imp." % col[-1]])] = temp
        #ret[tuple([c for c in col[:-1]] + ["%d imp." % i])] = temp
        
    return pd.DataFrame(ret)

def max_imp(df):
    ret = OrderedDict()
    d = df.to_dict(into=OrderedDict)
    
    def add_new_col(col_list, ret):
        new_col = {}
        
        for attack_name in [col_list[0][0][1], col_list[1][0][1]]:
            temp = list(filter(lambda t: t[0][1] == attack_name, col_list))
            imps = []
            for c in temp:
                imps.append([float(v.replace("$", "")) if v != -1 else -1 for _, v in c[1].items()])
            imps = (np.array(imps).T).argmax(axis=1)

            new_col = {}
            new_col_imp = {}
            new_col_eps = {}
            pcol = temp[0][0]
            new_col_name = ("-".join(pcol[0].split("-")[:-1]), pcol[1])
            new_col_imp_name = ("-".join(pcol[0].split("-")[:-1]), ("%s imp." % pcol[1]))
            new_col_eps_name = ("-".join(pcol[0].split("-")[:-1]), ("%s $\\epsilon$" % pcol[1]))
            for i, idx in enumerate(imps):
                k, v = list(temp[idx][1].items())[i]
                new_col[k] = v
                k, v = list(temp[idx][2].items())[i]
                new_col_imp[k] = v 

                new_col_eps[k] = "$" + ("%.1f$" % (float(temp[idx][0][0].split("-")[-1]) * 0.01))[1:]

            ret[new_col_eps_name] = new_col_eps
            ret[new_col_name] = new_col
            ret[new_col_imp_name] = new_col_imp
    
    prev_col = None
    temp = []
    for i, (col, col_dict) in enumerate(d.items()):
        if i == 0 or i == 1:
            ret[col] = col_dict
            continue
            
        if len(temp) == 0:
            temp.append(col_dict)
        elif i % 2 == 1:
            temp[-1] = (prev_col, temp[-1], col_dict)
            if i == (len(d.items())-1):
                add_new_col(temp, ret)
        else:
            if col[0].split("-")[:-1] != prev_col[0].split("-")[:-1]:
                add_new_col(temp, ret)
                temp = [col_dict]
            else:
                temp.append(col_dict)
                
        prev_col = col
        
    return pd.DataFrame(ret)

def table_wrapper(exp_name, grid_params, columns, rows, obj='avg_pert', caption="",
                  combine_method=None, additionals=None):
    t = """
\\begin{table}[h!]
\\tiny
\\centering
\\setlength{\\tabcolsep}{1.5pt}
"""
    df = pd.DataFrame({})
    if combine_method is None:
        df = avg_pert_table(exp_name, grid_params, columns, rows, obj)
    else:
        dfs = []
        for i, g in enumerate(grid_params):
            print(i)
            dfs.append(avg_pert_table(exp_name, g, columns, rows, obj))
        dfs = [avg_pert_table(exp_name, g, columns, rows, obj) for g in grid_params]
        df = pd.concat(dfs, axis=combine_method)
    
    if additionals:
        for fn in additionals:
            df = fn(df)
    
    if 'dataset' in rows:
        df = dataset_stat_column(df, grid_param, columns, rows)
    t += df.to_latex(escape=False)
    t += """\\caption{%s}
\\label{table:%s_%s}
\\end{table}
""" % (caption, exp_name, obj)
    return t


In [15]:
_, exp_name, grid_param, _ = nn_k1_robustness()
avg_caption = "1NN average perturbation distance (Linf)"
table_str = table_wrapper(exp_name, grid_param, ['n_samples', 'n_features', 'n_classes', 'model', 'attack'], ['dataset'], caption=avg_caption, 
                          additionals=[cmp_ratio, max_imp])
table_str = re.sub(r"([a-zA-Z_0-9'-]+) imp\.", "imp.", table_str)
table_str = re.sub(r"([a-zA-Z_0-9'-]+) \$\\epsilon\$", "$\\epsilon$", table_str)
write_to_tex(table_str, exp_name + '_table.tex')

In [16]:
_, exp_name, grid_param, _ = nn_k3_robustness()
avg_caption = "3NN average perturbation distance (Linf)"
#table_str = table_wrapper(exp_name, grid_param, ['model', 'attack'], ['dataset'], caption=avg_caption, additionals=[cmp_ratio])
#table_str = re.sub("([a-zA-Z_0-9'-]*) imp\.", "imp.", table_str)
#write_to_tex(table_str, exp_name + '_table.tex')
table_str = table_wrapper(exp_name, grid_param, ['n_samples', 'n_features', 'n_classes', 'model', 'attack'], ['dataset'], caption=avg_caption, 
                          additionals=[cmp_ratio, max_imp])
table_str = re.sub(r"([a-zA-Z_0-9'-]+) imp\.", "imp.", table_str)
table_str = re.sub(r"([a-zA-Z_0-9'-]+) \$\\epsilon\$", "$\\epsilon$", table_str)
write_to_tex(table_str, exp_name + '_table.tex')

In [6]:
_, exp_name, grid_param, _ = optimality()
avg_caption = "3NN and RF average perturbation distance (Linf)"
table_str = table_wrapper(exp_name, grid_param,
                          ['n_samples', 'n_features', 'n_classes', 'model', 'attack'], ['dataset'],
                          caption=avg_caption, combine_method=1)
write_to_tex(table_str, exp_name + '_table.tex')

0
1
2
3


ValueError: not enough values to unpack (expected 2, got 0)

In [7]:
_, exp_name, grid_param, _ = rf_robustness()
avg_caption = "Random forest average perturbation distance (Linf)"
table_str = table_wrapper(exp_name, grid_param, ['model', 'attack'], ['dataset'], caption=avg_caption, 
                          additionals=[cmp_ratio, max_imp])
table_str = re.sub(r"([a-zA-Z_0-9'-]+) imp\.", "imp.", table_str)
table_str = re.sub(r"([a-zA-Z_0-9'-]+) \$\\epsilon\$", "$\\epsilon$", table_str)
write_to_tex(table_str, exp_name + '_table.tex')

In [10]:
_, exp_name, grid_param, _ = dt_robustness()
avg_caption = "Decision tree average perturbation distance (Linf)"
table_str = table_wrapper(exp_name, grid_param, ['model', 'attack'], ['dataset'], caption=avg_caption, 
                          additionals=[cmp_ratio, max_imp])
table_str = re.sub(r"([a-zA-Z_0-9'-]+) imp\.", "imp.", table_str)
table_str = re.sub(r"([a-zA-Z_0-9'-]+) \$\\epsilon\$", "$\\epsilon$", table_str)
write_to_tex(table_str, exp_name + '_table.tex')

In [ ]:
_, _, grid_param, _ = nn_k1_robustness()
datasets = union_param_key(grid_param, 'dataset')
exp_name = "dataset-stats"
grid_param = {
    "dataset": datasets,
}
caption = "Data set statistics"
table_str = table_wrapper(exp_name, grid_param, ['n_samples', 'n_features', 'n_classes'], ['dataset'], caption=caption)
write_to_tex(table_str, exp_name + '_table.tex')

In [ ]:
_, exp_name, grid_param, _ = compare_nns()

def compare_nn_plots(exp_name, grid_param, caption='', show_plot=True):
    df = params_to_dataframe(grid_param)
    datasets = set.union(*[set(g['dataset']) for g in grid_param]) if isinstance(grid_param, list) else grid_param['dataset']

    control = {
        'dataset': datasets,
        'ord': grid_param[0]['ord'],
    }
    variables = ['model']
    plot_result(df, exp_name, control, variables, show_plot)
    return result_latex_figs(exp_name, control, caption)
fig_str = compare_nn_plots(exp_name, grid_param, show_plot=True)
write_to_tex(fig_str, exp_name + '_fig.tex')

In [17]:
%%bash
bash ./sync_report.sh

In [ ]:
experiments = [rf_attack, opt_of_rf_attack, robust_rf]
avg_caption = "average purturbation distance (Linf)"
miss_caption = "\\# of data algorithms is not able to generate successful attack (total 100 data points)"
for fn in experiments:
    _, exp_name, grid_param, _ = fn()
    print(exp_name)
    #columns = ['blackbox', 'rf_attack_rev_20', 'rf_attack_rev_100']
    table_str = table_wrapper(exp_name, grid_param, ['attack'], ['dataset'], caption=avg_caption)
    write_to_tex(table_str, exp_name + '_table.tex')
    fig_str = knn_attack_plots(exp_name, grid_param, show_plot=False)
    write_to_tex(fig_str, exp_name + '_fig.tex')

In [ ]:
nn_experiments = [nn_k1, nn_k3, nn_k5, nn_k7, opt_of_nnopt]
avg_caption = "average purturbation distance (Linf)"
miss_caption = "\\# of data algorithms is not able to generate successful attack (total 100 data points)"
for fn in nn_experiments:
    _, exp_name, grid_param, _ = fn()
    print(grid_param)
    variables = grid_param[0]['attack']
    #variables = list(filter(lambda v: 'kernelsub' not in v and 'direct' not in v, variables))
    variables = list(filter(lambda v: 'kernelsub' not in v, variables))
    print(variables)
    table_str = table_wrapper(exp_name, grid_param, variables, caption=avg_caption)
    table_str += table_wrapper(exp_name, grid_param, variables, obj='missed_count', caption=miss_caption)
    #print(table_str)
    write_to_tex(table_str, exp_name + '_table.tex')

In [ ]:
nn_experiments = [nn_k1, nn_k3, nn_k5, nn_k7, opt_of_nnopt]
for fn in nn_experiments:
    _, exp_name, grid_param, _ = fn()
    print(exp_name)
    fig_str = knn_attack_plots(exp_name, grid_param, show_plot=False)
    write_to_tex(fig_str, exp_name + '_fig.tex')

In [ ]:
nn_experiments = [robust_nn_k1, robust_nn_k3]
avg_caption = "average purturbation distance (Linf)"
miss_caption = "\\# of data algorithms is not able to generate successful attack (total 100 data points)"
for fn in nn_experiments:
    _, exp_name, grid_param, _ = fn()
    variables = grid_param[0]['attack']
    variables = list(filter(lambda v: 'kernelsub' not in v and 'direct' not in v, variables))
    print(variables)
    table_str = table_wrapper(exp_name, grid_param, variables, caption=avg_caption)
    table_str += table_wrapper(exp_name, grid_param, variables, obj='missed_count', caption=miss_caption)
    write_to_tex(table_str, exp_name + '_table.tex')

In [ ]:
nn_experiments = [robust_nn_k1, robust_nn_k3]
avg_caption = "average purturbation distance (Linf)"
miss_caption = "\\# of data algorithms is not able to generate successful attack (total 100 data points)"
for fn in nn_experiments:
    _, exp_name, grid_param, _ = fn()
    variables = grid_param[0]['attack']
    variables = list(filter(lambda v: 'kernelsub' not in v and 'direct' not in v, variables))
    print(variables)
    table_str = table_wrapper(exp_name, grid_param, variables, caption=avg_caption)
    table_str += table_wrapper(exp_name, grid_param, variables, obj='missed_count', caption=miss_caption)
    write_to_tex(table_str, exp_name + '_table.tex')